In [ ]:
URL= 'https://finance.yahoo.com/quote/IBM/'
headers={"User-Agent":"Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8"}

In [ ]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd

In [ ]:
session=requests.session()
response=session.get(URL,headers=headers)
soup=BeautifulSoup(response.text,"html.parser")
divs = soup.findAll("div",{"class":"D(ib) Mend(20px)"})
tds = soup.findAll("td",{"data-test":"TD_VOLUME-value"})


for td in tds:
    print(td.find('span').text)


for div in divs:
    print(div.find('span').text)

In [ ]:
import pandas as pd
IBMTest = pd.read_csv('./test/IBMTest.csv')
SentimentTest =pd.read_csv('./test/sentiments.csv')
SentimentTest = SentimentTest.loc[:, ['DateTime','Polarity']].groupby(['DateTime']).mean().reset_index()
SentimentTest['DateTime'] = pd.to_datetime(SentimentTest['DateTime'])
IBMTest['DateTime'] = pd.to_datetime(IBMTest['Date'].apply(str)+' '+IBMTest['Time'])
IBMTest.drop(columns=['Date','Time'],inplace = True)
IBMTest = IBMTest.set_index('DateTime')\
          .between_time('09:00:00', '16:00:00')\
          .reset_index()
print(len(IBMTest))

In [ ]:
import time
import socket
TCP_IP = '127.0.0.1'
TCP_PORT = 5000


s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((TCP_IP, TCP_PORT))
print("Awaiting Connection")

s.listen(1)
print("Server started at port 5000.")
while True:
    conn, addr = s.accept()
    print("Connected to: ", addr)
    for i in range(0,len(IBMTest)):
        IBMTest1 = IBMTest[i:i+1]
        Stocks_Data =pd.merge_asof(IBMTest1, SentimentTest.sort_values('DateTime'),
                      on='DateTime',
                      tolerance=pd.Timedelta('7m'))
        Stocks_Data.drop(columns=['High','Low','Close'],inplace=True)
        Stocks_Data.fillna(0,inplace=True)
        Stocks_Data['DateTime']= Stocks_Data['DateTime'].dt.strftime('%Y-%m-%d %H:%M:%S')
        Stocks_Data.reset_index(drop=True, inplace=True)
        print(Stocks_Data.reset_index().to_json(orient='records'))
        conn.send((Stocks_Data.to_json(orient='records')+'\n').encode())
        time.sleep(30)
    time.sleep(10)
    conn.close()